In [1]:
import json
import pandas as pd
import numpy as np
import os, sys
import re

In [2]:
# load all files in the folder
file = '.../phone_number_classified/crawl_data/data'
dirs = os.listdir(file)
print(len(dirs))
print(dirs[:2])

14
['sim_ngu_quy.json', 'sim_tu_quy.json']


In [3]:
ls_data = []
for file in dirs:
    print(file)
    keys_file = open(".../phone_number_classified/crawl_data/data/{}".format(file))
    keys = keys_file.read().encode('utf-8')
    keys_json = json.loads(keys)
    df = pd.DataFrame.from_dict(keys_json)
    ls_data.append(df)

sim_ngu_quy.json
sim_tu_quy.json
sim_add_01.json
sim_ong_dia.json
sim_thuong_2.json
sim_thuong1.json
sim_luc_quy.json
sim_thuong_00.json
sim2.json
sim_thuong_3.json
sim_add_02.json
sim_loc_phat.json
sim_than_tai.json
sim_thuong_03.json


In [4]:
df_data = pd.concat(ls_data)
df_data = df_data.dropna()
df_data.phone_number = df_data.phone_number.astype('str')
df_data.price = df_data.price.astype('str')
df_data.head(1)

,phone_number,price
0,0997188888,"Giá: 517,040,000"


# preprocessing data

In [5]:
def normlize_number_sequence(num_sequence):
    # convert tất cả những ký tự lớn hơn 2 khoảng trắng trởlên thành 1 khoảng trắng
    #num_sequence = re.sub(r'\s+', ' ', num_sequence)
    num_sequence = (num_sequence.lower()).strip()
    # xóa tất cả ký tự không phải là số
    sequence = re.sub(r'\D','', num_sequence)
    return  sequence.strip()
df_data['price_1'] = df_data.price.apply(normlize_number_sequence)
df_data['phone_number_1'] = df_data.phone_number.apply(normlize_number_sequence)
df_data.head(1)

,phone_number,price,price_1,phone_number_1
0,0997188888,"Giá: 517,040,000",517040000,0997188888


In [6]:
df_data.price_1 = df_data.price_1.astype('float')
df_data = df_data.drop(['phone_number', 'price'], axis= 1)

In [ ]:
#  create a dictionary, convert dau so di dong
convert_dau_so = dict()
convert_dau_so['162'] = '32'
convert_dau_so['163'] = '33'
convert_dau_so['164'] = '34'
convert_dau_so['165'] = '35'
convert_dau_so['166'] = '36'
convert_dau_so['167'] = '37'
convert_dau_so['168'] = '38'
convert_dau_so['169'] = '39'
convert_dau_so['120'] = '70'
convert_dau_so['121'] = '79'
convert_dau_so['122'] = '77'
convert_dau_so['126'] = '76'
convert_dau_so['128'] = '78'
convert_dau_so['123'] = '83'
convert_dau_so['124'] = '84'
convert_dau_so['125'] = '85'
convert_dau_so['127'] = '81'
convert_dau_so['129'] = '82'
convert_dau_so['199'] = '59'

# normalize mobile number
def norm_phone(phone_number):
    '''normalize mobile number (str)'''
    if phone_number[0] == '0':
        phone = phone_number[1:]
    else:
        phone = phone_number
    if phone[:3] in convert_dau_so.keys():
        phone = convert_dau_so[phone[:3]] + phone[3:]
    return phone

In [ ]:
def norm_phone(phone_number):
    '''normalize phone number (str)'''
    phone_number = str(phone_number)
    if phone_number[0] == '0':
        phone_number = phone_number[1:]
    return phone_number
def count_char(phone_number):
    '''count the character amount, before normalize phone number'''
    phone_number = str(phone_number)
    return len(phone_number)

df_data['phone_number'] = df_data.phone_number_1.apply(norm_phone)
df_data['length_of_phone_number'] = df_data.phone_number.apply(count_char)
df_data = df_data.drop('phone_number_1', axis= 1)
df_data.head(2)

In [ ]:
df_data = df_data[df_data.length_of_phone_number == 9]
df_data = df_data.drop_duplicates(subset= ['phone_number'])
df_data = df_data.dropna()
df_data.columns = ['price', 'phone_number', 'length_of_phone_number']
df_data.head(2)

1. target
+ < 200.000 : 0
+ 200.000 - 499.000: 1
+ 500.000 - 999.000: 2
+ 1.000.000- 1.999.000: 3
+ 2.000.000- 4.999.000: 4
+ 5.000.000 - 9.999.000: 5
+ 10.000.000 - 19.999.000: 6
+ 20.000.000 - 49.999.000: 7
+ \> 50.000.000: 8

In [ ]:
# convert target 1
def convert_label(price):
    if price < 200000:
        return 0
    elif (price >= 200000) & (price < 500000):
        return 1
    elif (price >= 500000) & (price < 1000000):
        return 2
    elif (price >= 1000000) & (price < 2000000):
        return 3
    elif (price >= 2000000) & (price < 5000000):
        return 4
    elif (price >= 5000000) & (price < 10000000):
        return 5
    elif (price >= 10000000) & (price < 20000000):
        return 6
    elif (price >= 20000000) & (price < 50000000):
        return 7
    elif (price >= 50000000):
        return 8
    else:
        pass
df_data['target1'] = df_data.price.apply(convert_label)

1. new target
+ < 200.000 : 0
+ 200.000 - 1.499.000: 1
+ 1.500.000- 9.999.000: 2
+ 10.000.000 - 49.999.000: 3
+ \> 50.000.000: 4

In [ ]:
# convert target 2
def convert_label2(price):
    if price < 200000:
        return 0
    elif (price >= 200000) & (price < 1500000):
        return 1
    elif (price >= 1500000) & (price < 10000000):
        return 2
    elif (price >= 10000000) & (price < 50000000):
        return 3
    elif (price >= 50000000):
        return 4
    else:
        pass
df_data['target2'] = df_data.price.apply(convert_label2)

In [ ]:
df_data.target2 = df_data.target2.astype('int')

In [ ]:
df_data = df_data.reset_index()
df_data = df_data.drop('index', axis= 1)

In [ ]:
df_data.target1.value_counts()

In [ ]:
df_data.target2.value_counts()

In [ ]:
df_data.columns

In [ ]:
df_data = df_data.reset_index()
df_data.columns = ['id','price', 'phone_number', 'length_of_phone_number', 'target1', 'target2']

In [ ]:
df_data.shape

# split train test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
idxs = df_data.id.values
targets = df_data.target2.values
id_train, id_test, target_train, target_test = train_test_split(idxs, targets, test_size = 0.2)

In [ ]:
df_train = df_data[df_data.id.isin(id_train)]
df_train = df_train.dropna()
df_train = df_train.reset_index()
df_train = df_train.drop(['index', 'id', 'length_of_phone_number'], axis= 1)
print(df_train.shape)
df_test = df_data[~df_data.id.isin(id_train)]
df_test = df_test.dropna()
df_test = df_test.reset_index()
df_test = df_test.drop(['index', 'id', 'length_of_phone_number'], axis= 1)
print(df_test.shape)

In [ ]:
df_test = df_test.reset_index()
df_test.columns = ['id_phone_test','price', 'phone_number', 'target1', 'target2']
df_train = df_train.reset_index()
df_train.columns = ['id_phone_train','price', 'phone_number', 'target1', 'target2']
df_train.head(1)

In [ ]:
df_train.target2 = df_train.target2.astype('int')
df_test.target2 = df_test.target2.astype('int')

In [ ]:
writer = pd.ExcelWriter('train_data.xlsx', engine='xlsxwriter')
df_train.to_excel(writer, 'Sheet1', encoding= 'utf-8', index= False)
writer.save()

In [ ]:
writer = pd.ExcelWriter('test_data.xlsx', engine='xlsxwriter')
df_test.to_excel(writer, 'Sheet1', encoding= 'utf-8', index= False)
writer.save()

In [ ]:
# df_train = pd.ExcelFile('train_data.xlsx', encoding= 'utf-8').parse('Sheet1')

In [ ]:
# df_train